# Input

In [156]:
type_chart = '''
Normal:   X X X X X X X X X X X X R I X X R X
Fire:     X R R S X S X X X X X S R X R X S X
Water:    X S R R X X X X S X X X S X R X X X 
Grass:    X R S R X X X R S R X R S X R X R X
Electric: X X S R R X X X I S X X X X R X X X
Ice:      X R R S X R X X S S X X X X S X R X
Fighting: S X X X X S X R X R R R S I X S S R
Poison:   X X X S X X X R R X X X R R X X I S
Ground:   X S X R S X X S X I X R S X X X S X
Flying:   X X X S R X S X X X X S R X X X R X
Psychic:  X X X X X X S S X X R X X X X I R S
Bug:      X R X S X X R R X R S X X R X S R R
Rock:     X S X X X S R X R S X S X X X X R X
Ghost:    I X X X X X X X X X S X X S X R X X
Dragon:   X X X X X X X X X X X X X X S X R I
Dark:     X X X X X X R X X X S X X S X R X R
Steel:    X R R X R S X X X X X X S X X X R S
Fairy:    X R X X X X S R X X X X X X S S R X
'''.strip()
   

In [157]:
print(type_chart.replace('X',' '))

Normal:                           R I     R  
Fire:       R R S   S           S R   R   S  
Water:      S R R         S       S   R       
Grass:      R S R       R S R   R S   R   R  
Electric:     S R R       I S         R      
Ice:        R R S   R     S S         S   R  
Fighting: S         S   R   R R R S I   S S R
Poison:         S       R R       R R     I S
Ground:     S   R S     S   I   R S       S  
Flying:         S R   S         S R       R  
Psychic:              S S     R         I R S
Bug:        R   S     R R   R S     R   S R R
Rock:       S       S R   R S   S         R  
Ghost:    I                   S     S   R    
Dragon:                               S   R I
Dark:                 R       S     S   R   R
Steel:      R R   R S             S       R S
Fairy:      R         S R             S S R  


In [158]:
labels = []
data = []
for line in type_chart.split('\n'):
    label, datum = line.split(':')
    labels.append(label)
    data.append(datum.strip().split())

## Simple Problem: Most / Least Weaknesses

In [159]:
def get_weaknesses(line):
    types = [labels[i] for i,x in enumerate(line) if x == 'S']
    return len(types), f'{len(types)} ({", ".join(types)})'

In [160]:
defensive_data = [list(i) for i in zip(*data)]

In [161]:
type_to_weaknesses = list(zip(labels, [get_weaknesses(x) for x in defensive_data]))

In [162]:
for type, (count, label) in sorted(type_to_weaknesses, key = lambda x:x[1][0], reverse=True):
    print(f'{type}: {label}')

Grass: 5 (Fire, Ice, Poison, Flying, Bug)
Rock: 5 (Water, Grass, Fighting, Ground, Steel)
Ice: 4 (Fire, Fighting, Rock, Steel)
Fire: 3 (Water, Ground, Rock)
Fighting: 3 (Flying, Psychic, Fairy)
Ground: 3 (Water, Grass, Ice)
Flying: 3 (Electric, Ice, Rock)
Psychic: 3 (Bug, Ghost, Dark)
Bug: 3 (Fire, Flying, Rock)
Dragon: 3 (Ice, Dragon, Fairy)
Dark: 3 (Fighting, Bug, Fairy)
Steel: 3 (Fire, Fighting, Ground)
Fairy: 3 (Poison, Psychic, Steel)
Water: 2 (Grass, Electric)
Poison: 2 (Ground, Psychic)
Ghost: 2 (Ghost, Dark)
Normal: 1 (Fighting)
Electric: 1 (Ground)


## Most offensive types

Simple example to start off: Fire, Water, Grass, Ice.

In [163]:
from numpy import linalg as LA

In [164]:
simple_matrix = [[0.5,0.5,2,2],
                 [2,0.5,0.5,1],
                 [0.5, 2, 0.5, 1],
                 [0.5, 0.5, 2, 0.5]]

In [165]:
ev = LA.eig(simple_matrix).eigenvectors

In [166]:
LA.eig(simple_matrix).eigenvalues[0]

(4.075960681792031+0j)

In [167]:
[round(x.real, 4) for x in np.matmul(simple_matrix,ev[:,0])]

[2.3492, 2.069, 1.9659, 1.7173]

OK, real example now.

In [168]:
CONSTANT_MAP = {'X': 1, 'R': 0.5, 'I': 0, 'S': 2}

In [169]:
matrix = np.array([[CONSTANT_MAP[x] for x in line] for line in data])

In [170]:
ev = LA.eig(matrix).eigenvectors

In [171]:
LA.eig(matrix).eigenvalues[0]

(18.772870315699883+0j)

In [172]:
ans = [x.real  for x in np.matmul(matrix,ev[:,0])]
ans = ans / min(ans)
ans = [round(x,4) for x in ans]
ans

[1.0,
 1.2473,
 1.2417,
 1.1196,
 1.101,
 1.2497,
 1.2263,
 1.0604,
 1.3189,
 1.2161,
 1.19,
 1.0911,
 1.2847,
 1.1701,
 1.0896,
 1.1582,
 1.2009,
 1.2201]

In [249]:
def analyze_offense0(ans, data, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    vals = [ans[j] * data[i][j] for j in range(len(ans))]
    top3_ind = np.argpartition(vals,-3)[-3:]
    top3_ind_sort = reversed(top3_ind[np.argsort(np.array(vals)[top3_ind])])
    top3_types = ['{:<9}'.format(labels[x]+':') + '{:<6}'.format(str(vals[x])) for x in top3_ind_sort]
    return str_begin + ' ('+ ', '.join(top3_types) + ')'

In [250]:
a = ['S', 'S', 'R', 'X', 'S']

In [ ]:
def analyze_offense0(ans, data, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    vals = [ans[j] * data[i][j] for j in range(len(ans))]
    top3_ind = np.argpartition(vals,-3)[-3:]
    top3_ind_sort = reversed(top3_ind[np.argsort(np.array(vals)[top3_ind])])
    top3_types = ['{:<9}'.format(labels[x]+':') + '{:<6}'.format(str(vals[x])) for x in top3_ind_sort]
    return str_begin + ' ('+ ', '.join(top3_types) + ')'

In [349]:
def get_annotated_labels(indices, ans, data, matrix, labels, i, header):
    indices = np.array(indices)
    vals = np.array([ans[j] * matrix[i][j] for j in range(len(ans))])
    indices_sort = reversed(indices[np.argsort(vals[indices])]) if indices.size > 0 else []
    if not indices_sort:
        return ''
    return header + ': ' + ', '.join(labels[x]+'(' + str(ans[x]) + ')' for x in indices_sort)

In [350]:
def analyze_offense1(ans, data, matrix, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    strong_indices = np.array([i for i,x in enumerate(data[i]) if x == 'S'])
    resisted_indices = np.array([i for i,x in enumerate(data[i]) if x == 'R'])
    immune_indices = np.array([i for i,x in enumerate(data[i]) if x == 'I'])
    strong_labels=get_annotated_labels(strong_indices, ans, data, matrix, labels, i, 'Strong')
    resisted_labels=get_annotated_labels(resisted_indices, ans, data, matrix, labels, i, 'Resisted')
    immune_labels=get_annotated_labels(immune_indices, ans, data, matrix, labels, i, 'Immune')
    additional_info = [strong_labels, resisted_labels, immune_labels]
    additional_info = [x for x in additional_info if x]
    return str_begin + '  ' + '\n                  '.join(additional_info)
        

In [351]:
analysis = [(x, analyze_offense1(ans, data, matrix, labels, i)) for i, x in enumerate(ans)]
for _, s in sorted(analysis, reverse=True):
    print(s)

Ground:   1.3189  Strong: Rock(1.2847), Fire(1.2473), Steel(1.2009), Electric(1.101), Poison(1.0604)
                  Resisted: Grass(1.1196), Bug(1.0911)
                  Immune: Flying(1.2161)
Rock:     1.2847  Strong: Ice(1.2497), Fire(1.2473), Flying(1.2161), Bug(1.0911)
                  Resisted: Ground(1.3189), Fighting(1.2263), Steel(1.2009)
Ice:      1.2497  Strong: Ground(1.3189), Flying(1.2161), Grass(1.1196), Dragon(1.0896)
                  Resisted: Ice(1.2497), Fire(1.2473), Water(1.2417), Steel(1.2009)
Fire:     1.2473  Strong: Ice(1.2497), Steel(1.2009), Grass(1.1196), Bug(1.0911)
                  Resisted: Rock(1.2847), Fire(1.2473), Water(1.2417), Dragon(1.0896)
Water:    1.2417  Strong: Ground(1.3189), Rock(1.2847), Fire(1.2473)
                  Resisted: Water(1.2417), Grass(1.1196), Dragon(1.0896)
Fighting: 1.2263  Strong: Rock(1.2847), Ice(1.2497), Steel(1.2009), Dark(1.1582), Normal(1.0)
                  Resisted: Fairy(1.2201), Flying(1.2161), Psychic(1.19

## Most defensive types

In [386]:
CONSTANT_MAP = {'X': 1, 'R': 2, 'I': 3, 'S': 0.5}

In [387]:
data_t = list(map(list, zip(*data)))

In [388]:
matrix = np.array([[CONSTANT_MAP[x] for x in line] for line in data_t])

In [398]:
matrix

array([[1. , 1. , 1. , 1. , 1. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. ,
        3. , 1. , 1. , 1. , 1. ],
       [1. , 2. , 0.5, 2. , 1. , 2. , 1. , 1. , 0.5, 1. , 1. , 2. , 0.5,
        1. , 1. , 1. , 2. , 2. ],
       [1. , 2. , 2. , 0.5, 0.5, 2. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
        1. , 1. , 1. , 2. , 1. ],
       [1. , 0.5, 2. , 2. , 2. , 0.5, 1. , 0.5, 2. , 0.5, 1. , 0.5, 1. ,
        1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 1. , 2. , 1. , 1. , 1. , 0.5, 2. , 1. , 1. , 1. ,
        1. , 1. , 1. , 2. , 1. ],
       [1. , 0.5, 1. , 1. , 1. , 2. , 0.5, 1. , 1. , 1. , 1. , 1. , 0.5,
        1. , 1. , 1. , 0.5, 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.5, 0.5, 2. , 2. ,
        1. , 1. , 2. , 1. , 0.5],
       [1. , 1. , 1. , 2. , 1. , 1. , 2. , 2. , 0.5, 1. , 0.5, 2. , 1. ,
        1. , 1. , 1. , 1. , 2. ],
       [1. , 1. , 0.5, 0.5, 3. , 0.5, 1. , 2. , 1. , 1. , 1. , 1. , 2. ,
        1. , 1. , 1. , 1. , 1. ],
       [1. , 1. , 1. , 2. , 0.5, 0.5,

In [400]:
LA.eig(matrix)

EigResult(eigenvalues=array([20.80993747+0.j        ,  0.36784592+2.98810204j,
        0.36784592-2.98810204j, -2.20572725+0.j        ,
       -0.44691103+1.86692361j, -0.44691103-1.86692361j,
       -0.09446303+1.83392374j, -0.09446303-1.83392374j,
        0.72158845+1.57445004j,  0.72158845-1.57445004j,
       -0.79886929+0.j        , -0.10737215+1.03450855j,
       -0.10737215-1.03450855j,  1.98961422+0.j        ,
        1.13535514+0.37449818j,  1.13535514-0.37449818j,
        1.52647912+0.19055659j,  1.52647912-0.19055659j]), eigenvectors=array([[-0.22189878+0.00000000e+00j,  0.01403276-4.30764671e-02j,
         0.01403276+4.30764671e-02j, -0.63523417+0.00000000e+00j,
         0.00091745+1.28207541e-01j,  0.00091745-1.28207541e-01j,
         0.04318752+3.90912472e-02j,  0.04318752-3.90912472e-02j,
        -0.14567621-2.70880423e-04j, -0.14567621+2.70880423e-04j,
        -0.08163537+0.00000000e+00j, -0.23497578+2.05728068e-02j,
        -0.23497578-2.05728068e-02j,  0.45855903+0.000

In [399]:
ev = LA.eig(matrix).eigenvectors

In [417]:
ans = np.array([x.real  for x in np.matmul(matrix,ev[:,0])])
if ans[0] < 0:
    ans *= -1
ans = ans / min(ans)
ans = [round(x,4) for x in ans]
ans

[1.1851,
 1.3622,
 1.2807,
 1.1724,
 1.2565,
 1.0,
 1.165,
 1.3183,
 1.2393,
 1.2868,
 1.1006,
 1.162,
 1.1717,
 1.3647,
 1.2412,
 1.2256,
 1.7092,
 1.2727]

In [418]:
def get_annotated_labels(indices, ans, data, matrix, labels, i, header):
    indices = np.array(indices)
    vals = np.array([ans[j] * matrix[i][j] for j in range(len(ans))])
    indices_sort = reversed(indices[np.argsort(vals[indices])]) if indices.size > 0 else None
    if indices_sort is None:
        return ''
    return header + ': ' + ', '.join(labels[x]+'(' + str(ans[x]) + ')' for x in indices_sort)

In [419]:
def analyze_defense(ans, data, matrix, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    strong_indices = np.array([i for i,x in enumerate(data[i]) if x == 'S'])
    resisted_indices = np.array([i for i,x in enumerate(data[i]) if x == 'R'])
    immune_indices = np.array([i for i,x in enumerate(data[i]) if x == 'I'])
    strong_labels=get_annotated_labels(strong_indices, ans, data, matrix, labels, i, 'Strong')
    resisted_labels=get_annotated_labels(resisted_indices, ans, data, matrix, labels, i, 'Resisted')
    immune_labels=get_annotated_labels(immune_indices, ans, data, matrix, labels, i, 'Immune')
    additional_info = [strong_labels, resisted_labels, immune_labels]
    additional_info = [x for x in additional_info if x]
    return str_begin + '  ' + '\n                  '.join(additional_info)
        

In [420]:
analysis = [(x, analyze_defense(ans, data_t, matrix, labels, i)) for i, x in enumerate(ans)]
for _, s in sorted(analysis,reverse=True):
    print(s)

Steel:    1.7092  Strong: Fire(1.3622), Ground(1.2393), Fighting(1.165)
                  Resisted: Steel(1.7092), Flying(1.2868), Fairy(1.2727), Dragon(1.2412), Normal(1.1851), Grass(1.1724), Rock(1.1717), Bug(1.162), Psychic(1.1006), Ice(1.0)
                  Immune: Poison(1.3183)
Ghost:    1.3647  Strong: Ghost(1.3647), Dark(1.2256)
                  Resisted: Poison(1.3183), Bug(1.162)
                  Immune: Normal(1.1851), Fighting(1.165)
Fire:     1.3622  Strong: Water(1.2807), Ground(1.2393), Rock(1.1717)
                  Resisted: Steel(1.7092), Fire(1.3622), Fairy(1.2727), Grass(1.1724), Bug(1.162), Ice(1.0)
Poison:   1.3183  Strong: Ground(1.2393), Psychic(1.1006)
                  Resisted: Poison(1.3183), Fairy(1.2727), Grass(1.1724), Fighting(1.165), Bug(1.162)
Flying:   1.2868  Strong: Electric(1.2565), Rock(1.1717), Ice(1.0)
                  Resisted: Grass(1.1724), Fighting(1.165), Bug(1.162)
                  Immune: Ground(1.2393)
Water:    1.2807  Strong: Elec

### Real Analysis

In [609]:
type_chart = '''
Normal:   X X X X X X X X X X X X R I X X R X
Fire:     X R R S X S X X X X X S R X R X S X
Water:    X S R R X X X X S X X X S X R X X X 
Grass:    X R S R X X X R S R X R S X R X R X
Electric: X X S R R X X X I S X X X X R X X X
Ice:      X R R S X R X X S S X X X X S X R X
Fighting: S X X X X S X R X R R R S I X S S R
Poison:   X X X S X X X R R X X X R R X X I S
Ground:   X S X R S X X S X I X R S X X X S X
Flying:   X X X S R X S X X X X S R X X X R X
Psychic:  X X X X X X S S X X R X X X X I R X
Bug:      X R X S X X R R X R S X X R X S R R
Rock:     X S X X X S R X R S X S X X X X R X
Ghost:    I X X X X X X X X X S X X S X R X X
Dragon:   X X X X X X X X X X X X X X S X R I
Dark:     X X X X X X R X X X S X X S X R X R
Steel:    X R R X R S X X X X X X S X X X R S
Fairy:    X R X X X X S R X X X X X X S S R X
'''.strip()
   

In [610]:
labels = []
data = []
for line in type_chart.split('\n'):
    label, datum = line.split(':')
    labels.append(label)
    data.append(datum.strip().split())

In [611]:
OFFENSIVE_CONSTANT_MAP = {'X': 1, 'R': 0, 'I': -1, 'S': 2}

In [612]:
offensive_matrix = np.array([[OFFENSIVE_CONSTANT_MAP[x] for x in line] for line in data])

In [613]:
DEFENSIVE_CONSTANT_MAP = {'X': 1, 'R': 2, 'I': 3, 'S': 0}

In [614]:
data_t = list(map(list, zip(*data)))

In [615]:
defensive_matrix = np.array([[DEFENSIVE_CONSTANT_MAP[x] for x in line] for line in data_t])

In [616]:
sum_matrix = (offensive_matrix + defensive_matrix)/2

In [617]:
ev = LA.eig(matrix).eigenvectors

In [618]:
ans = np.array([x.real  for x in np.matmul(sum_matrix,ev[:,0])])
if ans[0] < 0:
    ans *= -1
ans = ans / min(ans)
ans = [round(x,4) for x in ans]

In [655]:
def get_explanations(indices, ans, matrix, labels, i, header):
    vals = np.array([ans[j] * matrix[i][j] for j in range(len(ans))])
    return [(vals[idx]-ans[idx], header+labels[idx]+'(' + str(ans[idx])+')') for idx in indices]
    #return [(vals[idx]-ans[idx], header+labels[idx]+'(' + str(ans[idx])+')'+' value:'+str(vals[idx]-ans[idx])) for idx in indices]

In [659]:
def get_highlights(ans, data, data_t, offensive_matrix, defensive_matrix, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    strong_indices = np.array([i for i,x in enumerate(data[i]) if x == 'S'])
    resisted_indices = np.array([i for i,x in enumerate(data[i]) if x == 'R'])
    immune_indices = np.array([i for i,x in enumerate(data[i]) if x == 'I'])
    strong_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'S'])
    resisted_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'R'])
    immune_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'I'])
    good_explanations = []
    bad_explanations = []
    good_explanations.extend(get_explanations(strong_indices, ans, offensive_matrix, labels, i, 'Strong against '))
    good_explanations.extend(get_explanations(resisted_indices2, ans, defensive_matrix, labels, i, 'Resists '))
    good_explanations.extend(get_explanations(immune_indices2, ans, defensive_matrix, labels, i, 'Immune to '))
    bad_explanations.extend(get_explanations(strong_indices2, ans, defensive_matrix, labels, i, 'Weak to '))
    bad_explanations.extend(get_explanations(resisted_indices, ans, offensive_matrix, labels, i, 'Resisted by '))
    bad_explanations.extend(get_explanations(immune_indices, ans, offensive_matrix, labels, i, 'No effect against '))
    good_explanations.sort(reverse=True)
    bad_explanations.sort()
    good_explanations = good_explanations[:4]
    bad_explanations = bad_explanations[:4]
    return str_begin + '   Highlights: ' + ', '.join(x[1] for x in good_explanations)+ \
                       '\n' + (' ' * 19) + 'Lowlights:  ' + ', '.join(x[1] for x in bad_explanations)
    '''return str_begin + '   Highlights: ' + ('\n' + ' '*31).join(x[1] for x in good_explanations)+ \
                       '\n' + (' ' * 19) + 'Lowlights:  ' + ('\n' + ' '*31).join(x[1] for x in bad_explanations)'''

In [660]:
analysis = [(x, get_highlights(ans, data, data_t, offensive_matrix, defensive_matrix, labels, i)) for i, x in enumerate(ans)]
for _, s in sorted(analysis,reverse=True):
    print(s + "\n")

Steel:    1.4113   Highlights: Immune to Poison(1.1155), Resists Steel(1.4113), Strong against Fairy(1.2348), Resists Fairy(1.2348)
                   Lowlights:  Resisted by Steel(1.4113), Resisted by Fire(1.2799), Weak to Fire(1.2799), Weak to Ground(1.2664)

Fire:     1.2799   Highlights: Strong against Steel(1.4113), Resists Steel(1.4113), Resists Fire(1.2799), Resists Fairy(1.2348)
                   Lowlights:  Resisted by Fire(1.2799), Weak to Ground(1.2664), Resisted by Water(1.2545), Weak to Water(1.2545)

Ground:   1.2664   Highlights: Immune to Electric(1.1612), Strong against Steel(1.4113), Strong against Fire(1.2799), Strong against Electric(1.1612)
                   Lowlights:  No effect against Flying(1.226), Weak to Water(1.2545), Weak to Ice(1.0502), Resisted by Bug(1.0135)

Ghost:    1.2664   Highlights: Immune to Fighting(1.1061), Immune to Normal(1.0704), Strong against Ghost(1.2664), Resists Poison(1.1155)
                   Lowlights:  No effect against Normal(1.

In [578]:
def get_annotated_labels(indices, ans, matrix, labels, i, header):
    indices = np.array(indices)
    vals = np.array([ans[j] * matrix[i][j] for j in range(len(ans))])
    indices_sort = reversed(indices[np.argsort(vals[indices])]) if indices.size > 0 else None
    if indices_sort is None:
        return ''
    return header + ': ' + ', '.join(labels[x]+'(' + str(ans[x]) + ')' for x in indices_sort)

In [574]:
def analyze_sum(ans, data, data_t, matrix, labels, i):
    str_begin = '{:<9}'.format(labels[i]+':') + ' ' + '{:<6}'.format(str(ans[i]))
    strong_indices = np.array([i for i,x in enumerate(data[i]) if x == 'S'])
    resisted_indices = np.array([i for i,x in enumerate(data[i]) if x == 'R'])
    immune_indices = np.array([i for i,x in enumerate(data[i]) if x == 'I'])
    strong_labels=get_annotated_labels(strong_indices, ans, matrix, labels, i, 'Super Effective')
    resisted_labels=get_annotated_labels(resisted_indices, ans, matrix, labels, i, 'Resisted')
    immune_labels=get_annotated_labels(immune_indices, ans, matrix, labels, i, 'Immune')
    strong_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'S'])
    resisted_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'R'])
    immune_indices2 = np.array([i for i,x in enumerate(data_t[i]) if x == 'I'])
    strong_labels2=get_annotated_labels(strong_indices2, ans, sum_matrix, labels, i, 'Weak')
    resisted_labels2=get_annotated_labels(resisted_indices2, ans, sum_matrix, labels, i, 'Resists')
    immune_labels2=get_annotated_labels(immune_indices2, ans, sum_matrix, labels, i, 'Immune Against')
    additional_info = [strong_labels, resisted_labels, immune_labels, strong_labels2, resisted_labels2, immune_labels2]
    additional_info = [x for x in additional_info if x]
    return str_begin + '  ' + '\n                  '.join(additional_info)

In [575]:
analysis = [(x, analyze_sum(ans, data, data_t, sum_matrix, labels, i)) for i, x in enumerate(ans)]
for _, s in sorted(analysis,reverse=True):
    print(s)

Ground:   2.514   Super Effective: Rock(2.0204), Fire(2.1803), Steel(1.9909), Electric(1.5861), Poison(1.5195)
                  Resisted: Bug(1.6298), Grass(1.6549)
                  Immune: Flying(1.8285)
                  Weak: Ice(1.9454), Water(1.9085), Grass(1.6549)
                  Resists: Rock(2.0204), Poison(1.5195)
                  Immune Against: Electric(1.5861)
Fighting: 2.2615  Super Effective: Rock(2.0204), Steel(1.9909), Ice(1.9454), Dark(1.5798), Normal(1.0)
                  Resisted: Bug(1.6298), Poison(1.5195), Fairy(1.8626), Psychic(1.4794), Flying(1.8285)
                  Immune: Ghost(1.6644)
                  Weak: Fairy(1.8626), Psychic(1.4794), Flying(1.8285)
                  Resists: Rock(2.0204), Dark(1.5798), Bug(1.6298)
Fire:     2.1803  Super Effective: Steel(1.9909), Ice(1.9454), Grass(1.6549), Bug(1.6298)
                  Resisted: Fire(2.1803), Dragon(1.3055), Rock(2.0204), Water(1.9085)
                  Weak: Ground(2.514), Rock(2.0204), Water(